### ***User Based collaborative filtering using svd***

**Definition-**collaborative filtering is a method to predict a rating for a user item pair based on the history of ratings given by the user and given to the item

**Why using svd?**

**Answer**- The SVD technique was introduced into the recommendation system domain by Brandyn Webb, much more famously known as Simon Funk during the Netflix Prize challenge.Conceptually it makes sense that an SVD based model is very well fit to a recommender system since a large strength of SVD is its ability to establish the relatedness of items based on a set of factors known as latent factors. 

importing libraries

In [41]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
import numpy as np
from scipy import *

In [42]:
df=pd.read_csv('/content/C__Users_Nabeel-pc_Desktop_movies_dataframe_grouplens.csv')

In [46]:
df['MovieID']=df['MovieID'].apply(pd.to_numeric)

In [47]:
df.head()

,MovieID,Title,Genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [48]:
df.shape

(3883, 3)

In [49]:
df_users=pd.read_csv("/content/users.csv")

In [50]:
df_users.head()

,UserID,Gender,Age,Occupation,Zip-code
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,2460
4,5,M,25,20,55455


In [51]:
df_users.shape

(6040, 5)

In [52]:
df_ratings=pd.read_csv('/content/ratings.csv')

In [53]:
df_ratings.head()

,UserID,MovieID,Rating,Timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [54]:
df_ratings.shape

(1000209, 4)

***user based collaborative filtering without using surprise library***

In [55]:
df_movie_features = df_ratings.pivot(index='UserID',columns='MovieID',values='Rating').fillna(0)

In [56]:
df_movie_features.head()


MovieID,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,3913,3914,3915,3916,3917,3918,3919,3920,3921,3922,3923,3924,3925,3926,3927,3928,3929,3930,3931,3932,3933,3934,3935,3936,3937,3938,3939,3940,3941,3942,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
UserID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,4.0,0.0,4.0,0.0,3.0,0.0,0.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [57]:
import numpy as np
#R = df_movie_features.as_matrix()
R=np.asarray(df_movie_features)
user_ratings_mean = np.mean(R, axis = 1)
R_demeaned = R - user_ratings_mean.reshape(-1, 1)
print(R_demeaned)


[[ 4.94009714 -0.05990286 -0.05990286 ... -0.05990286 -0.05990286
  -0.05990286]
 [-0.12924987 -0.12924987 -0.12924987 ... -0.12924987 -0.12924987
  -0.12924987]
 [-0.05369671 -0.05369671 -0.05369671 ... -0.05369671 -0.05369671
  -0.05369671]
 ...
 [-0.02050729 -0.02050729 -0.02050729 ... -0.02050729 -0.02050729
  -0.02050729]
 [-0.1287102  -0.1287102  -0.1287102  ... -0.1287102  -0.1287102
  -0.1287102 ]
 [ 2.6708041  -0.3291959  -0.3291959  ... -0.3291959  -0.3291959
  -0.3291959 ]]


In [58]:
from scipy.sparse.linalg import svds
U, sigma, Vt = svds(R_demeaned, k = 50)
# that the Sigma$ returned is just the values instead of a diagonal matrix. 
# This is useful, but since I'm going to leverage matrix multiplication to get predictions 
# I'll convert it to the diagonal matrix form.
sigma = np.diag(sigma)
all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) + user_ratings_mean.reshape(-1, 1)
print(all_user_predicted_ratings)

[[ 4.28886061  0.14305516 -0.1950795  ...  0.03191195  0.05044975
   0.08891033]
 [ 0.74471587  0.16965927  0.33541808 ... -0.10110207 -0.0540982
  -0.14018846]
 [ 1.81882382  0.45613623  0.09097801 ...  0.01234452  0.01514752
  -0.10995596]
 ...
 [ 0.61908871 -0.16176859  0.10673806 ... -0.01336948 -0.0303543
  -0.11493552]
 [ 1.50360483 -0.03620761 -0.16126817 ... -0.01090407 -0.03864749
  -0.16835943]
 [ 1.99624816 -0.18598715 -0.1564782  ... -0.00664061  0.12706713
   0.28500112]]


In [59]:
preds_df = pd.DataFrame(all_user_predicted_ratings, columns = df_movie_features.columns)
preds_df.head()

MovieID,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,3913,3914,3915,3916,3917,3918,3919,3920,3921,3922,3923,3924,3925,3926,3927,3928,3929,3930,3931,3932,3933,3934,3935,3936,3937,3938,3939,3940,3941,3942,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
0,4.288861,0.143055,-0.195080,-0.018843,0.012232,-0.176604,-0.074120,0.141358,-0.059553,-0.195950,0.512867,-0.089172,0.310181,-0.002005,-0.052401,-0.189827,0.238360,0.006466,-0.099315,-0.069682,-0.321492,0.111577,0.034795,0.320576,-0.118217,-0.012647,0.065573,-0.098318,0.064081,-0.005914,0.091936,0.180563,-0.009566,2.641693,-0.012495,0.765179,0.019784,0.002917,0.053079,0.014856,...,0.018810,-0.018782,0.022249,0.227852,-0.067653,-0.046039,-0.023574,-0.019405,-0.005116,-0.032921,-0.008259,-0.019157,0.007527,-0.008687,-0.025630,-0.013563,0.015240,-0.044665,-0.009568,-0.043549,-0.003131,-0.008221,-0.005948,0.031885,-0.003424,-0.001159,-0.002124,-0.002827,0.010393,-0.001068,0.027807,0.001640,0.026395,-0.022024,-0.085415,0.403529,0.105579,0.031912,0.050450,0.088910
1,0.744716,0.169659,0.335418,0.000758,0.022475,1.353050,0.051426,0.071258,0.161601,1.567246,0.772656,0.046179,-0.054562,0.042344,0.048390,0.347313,1.074905,-0.099782,0.008163,0.250869,2.186638,0.018789,-0.002199,0.218934,0.824475,0.139274,-0.007135,0.053071,-0.156952,0.044739,-0.002960,0.453298,-0.007484,0.920325,0.016566,1.335129,-0.015066,-0.045602,0.034649,0.122010,...,-0.042363,-0.137822,-0.112071,0.380783,-0.036273,-0.016174,0.002920,-0.148021,-0.017614,-0.033474,0.086133,0.008153,-0.126819,0.109208,0.001798,0.151866,0.014118,0.032897,0.005764,0.042259,0.022404,0.003260,0.010556,0.137181,-0.042184,0.006759,-0.005789,0.000340,0.002024,0.016013,-0.056502,-0.013733,-0.010580,0.062576,-0.016248,0.155790,-0.418737,-0.101102,-0.054098,-0.140188
2,1.818824,0.456136,0.090978,-0.043037,-0.025694,-0.158617,-0.131778,0.098977,0.030551,0.735470,-0.023476,0.034796,0.065942,0.008661,0.110348,-0.002952,-0.122061,0.063974,0.061033,0.081799,0.329471,0.149579,0.095352,-0.161493,0.022545,-0.009284,-0.002677,-0.142710,0.012345,-0.085331,0.076139,-0.355795,-0.008579,1.046871,-0.088946,0.383583,-0.018144,-0.038618,0.113984,0.006942,...,0.007233,-0.047221,0.066474,-0.179455,0.097428,0.034113,0.008098,-0.024784,-0.012749,-0.007394,-0.017220,0.004719,0.113348,-0.074943,-0.145795,0.128619,0.112567,0.045500,-0.018027,-0.058946,-0.002770,-0.035276,-0.008085,0.132182,-0.017005,0.014383,0.006598,-0.006217,-0.000342,0.000518,0.040481,-0.005301,0.012832,0.029349,0.020866,0.121532,0.076205,0.012345,0.015148,-0.109956
3,0.408057,-0.072960,0.039642,0.089363,0.041950,0.237753,-0.049426,0.009467,0.045469,-0.111370,-0.375831,0.068658,0.011199,0.069699,-0.037529,-0.238788,0.060607,-0.043418,0.053152,0.078237,0.357185,-0.096005,-0.028243,-0.067169,0.246164,-0.020379,0.034461,-0.022225,-0.012327,0.009182,0.014730,0.215893,-0.019687,-0.293933,-0.011511,0.145326,-0.029213,0.030029,-0.045409,-0.030684,...,-0.015077,-0.030208,0.028357,-0.072643,-0.135727,-0.053318,-0.012962,-0.054465,0.005870,-0.018048,-0.006836,-0.008222,-0.027214,-0.071677,-0.094072,-0.010745,-0.103191,-0.031297,-0.023920,-0.015053,-0.017914,-0.029561,-0.024299,-0.057678,-0.111450,-0.015473,-0.007123,-0.007416,-0.011508,-0.010038,0.008571,-0.005425,-0.008500,-0.003417,-0.083982,0.094512,0.057557,-0.026050,0.014841,-0.034224
4,1.574272,0.021239,-0.051300,0.246884,-0.032406,1.552281,-0.199630,-0.014920,-0.060498,0.450512,-0.251178,0.012337,-0.084051,0.258937,0.016570,0.980536,1.267869,0.275619,-0.008139,-0.038832,1.849627,0.107649,-0.168424,0.386541,1.790343,0.192379,-0.054356,0.267566,1.027817,0.374665,-0.010445,1.947980,0.017468,2.784035,0.274397,1.422393,0.040553,0.022926,1.345800,0.104507,...,0.075475,0.330767,0.150470,-0.261636,0.085163,-0.014229,-0.029247,0.124172,0.092875,0.061895,0.034757,0.054386,0.047055,0.048403,0.082926,0.129035,-0.174646,0.102727,0.024732,0.047280,0.017818,0.041451,0.041595,-0.007138,-0.080448,0.01

In [60]:
def recommend_movies(predictions_df, userID, df, original_ratings_df, num_recommendations=10):
  user_row_number = userID - 1 # UserID starts at 1, not 0
  sorted_user_predictions = preds_df.iloc[user_row_number].sort_values(ascending=False) # UserID starts at 1
  #print(sorted_user_predictions)
  #Get the user's data and merge in the movie information.
  user_data = original_ratings_df[original_ratings_df.UserID == (userID)]
  user_full = (user_data.merge(df, how = 'left', left_on = 'MovieID', right_on = 'MovieID').
                      sort_values(['Rating'], ascending=False))

  print ('User {0} has already rated {1} movies'.format(userID, user_full.shape[0]))
  #print( 'Recommending highest {0} predicted ratings movies not already rated.'.format(num_recommendations))
    
     # Recommend the highest predicted rating movies that the user hasn't seen yet.
  recommendations = (df[~df['MovieID'].isin(user_full['MovieID'])].merge(pd.DataFrame(sorted_user_predictions).reset_index(), how = 'left').rename(columns = {user_row_number: 'Predictions'}).
  sort_values('Predictions', ascending = False).iloc[:num_recommendations, :-1])
  return user_full, recommendations

In [61]:
a=int(input('Enter any userid: '))
already_rated, predictions = recommend_movies(preds_df, a, df, df_ratings, 10)
print("The movies that user has already rated",already_rated)
print('Recommend movies for userid',a, 'are following :')
predictions.head()


Enter any userid: 1
User 1 has already rated 53 movies
The movies that user has already rated     UserID  ...                                         Genres
0        1  ...                                          Drama
46       1  ...                   Animation|Children's|Musical
40       1  ...                    Animation|Children's|Comedy
18       1  ...                                          Drama
41       1  ...                                          Drama
23       1  ...                                      Drama|War
37       1  ...                   Animation|Children's|Musical
14       1  ...                                        Musical
25       1  ...           Animation|Children's|Musical|Romance
45       1  ...                      Children's|Comedy|Musical
10       1  ...                   Animation|Children's|Musical
22       1  ...                                  Comedy|Sci-Fi
36       1  ...                                          Drama
48       1  ...         

,MovieID,Title,Genres
311,318,"Shawshank Redemption, The (1994)",Drama
32,34,Babe (1995),Children's|Comedy|Drama
356,364,"Lion King, The (1994)",Animation|Children's|Musical
1975,2081,"Little Mermaid, The (1989)",Animation|Children's|Comedy|Musical|Romance
1235,1282,Fantasia (1940),Animation|Children's|Musical


***user based cf using surprise library***

In [62]:
pip install scikit-surprise

Mapping of movie titles

In [63]:
data_title = pd.read_csv('/content/C__Users_Nabeel-pc_Desktop_movies_dataframe_grouplens.csv')
data_title.set_index('MovieID', inplace = True)
print (data_title.head(10))

                                      Title                        Genres
MovieID                                                                  
1                          Toy Story (1995)   Animation|Children's|Comedy
2                            Jumanji (1995)  Adventure|Children's|Fantasy
3                   Grumpier Old Men (1995)                Comedy|Romance
4                  Waiting to Exhale (1995)                  Comedy|Drama
5        Father of the Bride Part II (1995)                        Comedy
6                               Heat (1995)         Action|Crime|Thriller
7                            Sabrina (1995)                Comedy|Romance
8                       Tom and Huck (1995)          Adventure|Children's
9                       Sudden Death (1995)                        Action
10                         GoldenEye (1995)     Action|Adventure|Thriller


Getting information about the movies that a given user 1 has rated 5

In [64]:
user = df_ratings[(df_ratings['UserID'] == 1) & (df_ratings['Rating'] == 5)]
user = user.set_index('MovieID')
user = user.join(data_title)['Title']
print(user)

MovieID
1193    One Flew Over the Cuckoo's Nest (1975)
2355                      Bug's Life, A (1998)
1287                            Ben-Hur (1959)
2804                 Christmas Story, A (1983)
595                Beauty and the Beast (1991)
1035                Sound of Music, The (1965)
3105                         Awakenings (1990)
1270                 Back to the Future (1985)
527                    Schindler's List (1993)
48                           Pocahontas (1995)
1836            Last Days of Disco, The (1998)
1022                         Cinderella (1950)
150                           Apollo 13 (1995)
1                             Toy Story (1995)
1961                           Rain Man (1988)
1028                       Mary Poppins (1964)
1029                              Dumbo (1941)
2028                Saving Private Ryan (1998)
Name: Title, dtype: object


In [65]:
from surprise import SVD
from surprise import Dataset,Reader
from surprise import accuracy
from collections import defaultdict
from surprise.model_selection import cross_validate,train_test_split
#reader tells our svd what the lower and upper bound of our rating is
reader=Reader(rating_scale=(1,5))

In [66]:
data1=df_ratings[:500000]

In [67]:
df_ratings.shape

(1000209, 4)

In [68]:
data=Dataset.load_from_df(df_ratings[['UserID','MovieID','Rating']],reader)

In [69]:
# Retrieve the trainset
trainset = data.build_full_trainset()

In [70]:
model=SVD(n_factors=100,biased=False) #n_factors are no of latent factors
#model.fit(trainset)

In [71]:
# Run 5-fold cross-validation and print results
a=cross_validate(model, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)
tmp = pd.DataFrame.from_dict(a).mean(axis=0)
tmp.head()

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8755  0.8724  0.8752  0.8751  0.8748  0.8746  0.0011  
MAE (testset)     0.6886  0.6867  0.6886  0.6881  0.6874  0.6879  0.0007  
Fit time          49.79   50.40   50.50   50.04   50.65   50.28   0.32    
Test time         2.59    2.45    3.00    2.14    2.59    2.55    0.28    


test_rmse     0.874616
test_mae      0.687876
fit_time     50.278176
test_time     2.554125
dtype: float64

In [72]:
model.fit(trainset)

''' use of build_anti_testset-  return a list of ratings that can be used as a testset in the test() method
the ratings are all ratings that are not in the testset'''
testset = trainset.build_anti_testset()

predictions = model.test(testset)
#predictions.predict(1,2)  #first parameter is raw user id while second is raw item id 


In [73]:
def predection( userid ):

  user = data_title.copy()
  user = user.reset_index()
  #user = user[~user['Movie_Id'].isin(drop_movie_list)]
  user['Estimate_Score'] = user['MovieID'].apply(lambda x: model.predict(userid, x).est)
  user = user.drop('MovieID', axis = 1)
  user= user.sort_values('Estimate_Score', ascending=False)
  x=print(user.head(10))
  return

In [74]:
df_ratings.head()

,UserID,MovieID,Rating,Timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [75]:
#make predection for the user 1
predection(1)

                                 Title  ... Estimate_Score
315   Shawshank Redemption, The (1994)  ...       4.878221
1022        Sound of Music, The (1965)  ...       4.847096
907           Wizard of Oz, The (1939)  ...       4.782339
523            Schindler's List (1993)  ...       4.738699
941       It's a Wonderful Life (1946)  ...       4.727634
1015               Mary Poppins (1964)  ...       4.670776
589   Silence of the Lambs, The (1991)  ...       4.624984
1892                   Rain Man (1988)  ...       4.617854
2735         Christmas Story, A (1983)  ...       4.599389
1189      To Kill a Mockingbird (1962)  ...       4.586677

[10 rows x 3 columns]


***Question***

Maths Behind SVD

SVD is a matrix factorization technique that is usually used to reduce the number of features of a data set by reducing space dimensions from N to K where K < N. 

In linear algebra, **the Singular value decomposition(SVD) ** is  factorisation of areal or complex matrix that generalizes the eigendecoposition of a square noral atrix to any m*n  matrix via an extension of the polar decomposition.

specifiacally,the singular value decompositon of an  m*n  real or complex matrix M is factorization of the form  
                         U*S*V(transpose)  
 where U is an m*m real or complex unitary matrix, 
 S is an m*n rectangular diagonal matrix with non-negative real numbers on     the diagonal , and 
 V is an n*n real or complex unitary matrix.if M is real ,U and V(Transpose) are real orthogonal  matrices.

The diagonal S(sigma) are known as singular values of M.The Columns of U and the Columns of V are called the left-singular vectors and right-singular vectors of M,respectively.


